# Enhacing Dynamic World for Active and Fallow Cropland Mapping

**By:** Jedidiah Chibinga

## Overview
This notebook implements a workflow to distinguish between **Active Cropland** (currently growing) and **Fallow Cropland** (agricultural land currently resting or abandoned). This distinction is critical for humanitarian food security assessments.

### Methodology
Uses **Dynamic World labels only** (no probabilities) and stabilise the time series by creating a **monthly modal label** image for each month (mode of all DW labels in that month).
1.  **Data Source:** Google Dynamic World (Sentinel-2 LULC probabilities).
2.  **Potential Cropland Mask:** Uses a 5-year historical frequency analysis to identify where agriculture *has* happened.
3.  **Active vs. Fallow Classification:** Checks the current state of pixels within that mask.
4.  **Duration Analysis:** Calculates how long a field has been fallow (Red intensity) or active (Green intensity).

#### Key parameters
- **Historical Crop Period**: long history period (e.g., 5 years)
- **Current Crop Period**: current period for status classification (e.g., 3 months)
- **Crop Window**: sliding window size and dominance threshold (e.g., 4 months window, at least 3 crop months)
- **Mnimum Crop Months**: minimum crop-months inside crop window to be considered croplands (e.g., 3 out of 4 months)

#### Outputs

This notebook maps:
- **Active Cropland / Fallow Classification**: 
    
    `Active cropland`- currently labelled as crops often enough during the current window, 
    
    `Fallow` - not currently crops, but historically sustained cropland
- **Active Cropland Duration**: crop-month count in last 12 months (0–12) for active pixels
- **Fallow Duration**: months since last crop-month consecutive croplands


In [1]:
# If modules are not installed, uncomment the install code line below:
# !pip -q install earthengine-api geemap

# Install/import packages
import ee
import geemap

# Authenticate/initialize Earth Engine
try:
    ee.Initialize()
except Exception as e:
    ee.Authenticate()
    ee.Initialize()

print("Earth Engine initialized.")


Successfully saved authorization token.
Earth Engine initialized.


## Define Functions

All functions to get final *image* outputs are defined here

In [2]:
# # ---- Active threshold within current window ----
# # For curr window = 3 months, A=2 works as a flicker-tolerant threshold
# A = 2

# # ---- Intensity settings ----
# active_int_months = 12  # months to count crop presence for active intensity
# fallow_cap_months = 24  # cap fallow intensity for visualization stability


### Load Dynamic World and build monthly modal label ImageCollection

Involves:
1. Loading `GOOGLE/DYNAMICWORLD/V1`
2. Filtering to aoi and history window
3. Building a **monthly** collection where each month is represented by the **mode** of all labels in that month.


In [3]:
# Get Dynamic World historical image collection 
def dw_historical_image_col(aoi, historical_start_date, historical_end_date):
    # Load Dynamic World
    DW = ee.ImageCollection("GOOGLE/DYNAMICWORLD/V1").select("label")

    # Filter to aoi + hist window (default: 5 years historical window)
    return  DW.filterBounds(aoi).filterDate(historical_start_date, historical_end_date)


# Generate list of month start dates between two ee.Dates
def month_starts(start_date, end_date):
    n_months = end_date.difference(start_date, "month").toInt()

    return ee.List.sequence(0, n_months.subtract(1)).map(
        lambda m: start_date.advance(ee.Number(m), "month")
    )


# Make monthly image mode
def make_mode(d, aoi, month_ic):
    mode_image = month_ic.reduce(ee.Reducer.mode())

    # Typically 'label_mode', but handle naming defensively
    bandnames = mode_image.bandNames()
    label_band = ee.String(
        ee.Algorithms.If(bandnames.contains("label_mode"), "label_mode", bandnames.get(0))
    )

    return (
        mode_image.select([label_band]).rename("label").toInt()
        .set("system:time_start", d.millis()).clip(aoi)
    )

def dw_historic_modal_image_col(aoi, historic_date_start, historic_date_end):
    historic_date_start = ee.Date(historic_date_start)
    historic_date_end = ee.Date(historic_date_end)
    
    historic_date_months = month_starts(historic_date_start, historic_date_end)
    
    # Build monthly modal label collection
    def monthly_mode_label(d):
        d = ee.Date(d)
        d_next = d.advance(1, "month")
        
        dw_historic_image_col = dw_historical_image_col(aoi, historic_date_start, historic_date_end)
        month_ic = dw_historic_image_col.filterDate(d, d_next)

        # If there are no images, return a fully-masked placeholder image
        empty_image = (
            ee.Image.constant(0)
            .rename("label")
            .updateMask(ee.Image(0))
            .toInt()
            .set("system:time_start", d.millis())
            .clip(aoi)
        )

        mode_image = make_mode(d, aoi, month_ic)

        return ee.Image(ee.Algorithms.If(month_ic.size().gt(0), mode_image, empty_image))
    
    return ee.ImageCollection.fromImages(historic_date_months.map(monthly_mode_label)) # dw_monthly

# print("Monthly DW collection size (historical):", dw_monthly.size().getInfo())

In [4]:
# # Example AOI
# # aoi = ee.Geometry.Rectangle([27.75120,-15.8775, 27.9214,-15.7321]) # Mazabuka
# # aoi = ee.Geometry.Rectangle([40.5121, 37.0099, 40.6247, 37.0736])
# # aoi = ee.Geometry.Rectangle([40.5121, 37.0099, 40.6247, 37.0736])

# bbox_long_lat1 = [27.9747, -15.7329]
# bbox_long_lat2 = [27.834, -15.8436]
# aoi = ee.Geometry.Rectangle(bbox_long_lat1 + bbox_long_lat2)

# # Long-term history window (e.g., 5 years)
# historic_date_start="2020-01-01"
# historic_date_end="2025-12-31"

# dw_image_col = dw_historical_image_col(aoi, historic_date_start, historic_date_end)
# dw_monthly_image_col = dw_historic_modal_image_col(aoi, historic_date_start, historic_date_end)

# print("DW collection size (historical):", dw_image_col.size().getInfo())
# print("Monthly DW collection size (historical):", dw_monthly_image_col.size().getInfo())

# m = geemap.Map(basemap='HYBRID')
# m.centerObject(aoi, 12)
# m.addLayer(dw_image_col.first(), {'min':0, 'max':8})
# m.addLayer(dw_monthly_image_col.first(), {'min':0, 'max':8})
# m

### Create a binary monthly crop mask

Creates a monthly binary time series:
- `is_crop = 1` if monthly label == 4
- `is_crop = 0` otherwise


In [5]:
# Monthly binary crop image collection
def binary_crop_image_col(aoi, historic_date_start, historic_date_end):
    dw_monthly = dw_historic_modal_image_col(aoi, historic_date_start, historic_date_end)
    
    # Create crop mask for each emage to be used in mapping function
    def binary_crop_mask(image):
        image = ee.Image(image)
        
        # Cropland label in Dynamic World
        CROP_LABEL = 4
        
        is_crop = image.eq(CROP_LABEL).rename("is_crop").toByte()
        return is_crop.copyProperties(image, ["system:time_start"])
    
    return dw_monthly.map(binary_crop_mask)

### Potential cropland mask from history (sliding window dominance)

A pixel is considered **potential cropland** if it satisfies:

>“Was a pixel **classified as crop** in at least `C`_(min_crop_months)_ months within any **`W`-month**_(crop_window)_ period?”

Approach:
- Convert crop_monthly to a list (one image per month)
- For each window start index i, sum images i..i+W-1
- Take the maximum window sum across all windows
- Potential cropland if `max_window_sum >= C`


In [6]:
# Get maximum number consecutive months a a pixel was cropland
def get_consecutive_crop_count(crop_monthly_col, crop_window):
    crop_col_size = crop_monthly_col.size()
    crop_list = crop_monthly_col.toList(crop_col_size)

    # Number of windows
    no_sliding_windows = crop_col_size.subtract(crop_window).add(1)

    # Get number of months/times a pixel was classified as cropland in the window
    def window_sum(i):
        i = ee.Number(i)
        window_imgs = ee.ImageCollection.fromImages(crop_list.slice(i, i.add(crop_window)))
        return window_imgs.sum().rename("win_sum").toInt()

    # Apply window sum to all the sliding windows and gives one image per window
    window_sums = ee.ImageCollection.fromImages(
        ee.List.sequence(0, no_sliding_windows.subtract(1)).map(window_sum)
    )

    # Get the maximum number of crop months across all windows
    return window_sums.max().rename("max_win_sum").toInt()

# Get the mask for potential cropland in the aoi
def potential_cropland_mask(aoi, historic_date_start, historic_date_end, crop_window, min_crop_months):
    crop_monthly_col = binary_crop_image_col(aoi, historic_date_start, historic_date_end)
    
    # Get maximum number consecutive months a a pixel was cropland
    max_window_sum = get_consecutive_crop_count(crop_monthly_col, crop_window)

    # Classify as cropland if pixel was cropland for more than minimum crop month per window
    potential_crop_land = (max_window_sum.gte(min_crop_months).rename("potential_cropland")
                           .toByte().clip(aoi))
    
    # Remove pixels other pixels
    return potential_crop_land.updateMask(potential_crop_land.neq(0))


### Current window classification for Active and Fallow 

Classify using **monthly mode labels**, then apply thresholds:

- `current_crop_months = sum(is_crop)` inside `Current` date period
- **Active** if `current_crop_months >= min_active_crop_months`
- **Fallow** if potential cropland historically, but not active now
<!-- - **Other** otherwise -->

Output class values:
- 2 = active
- 1 = fallow
<!-- - 0 = other -->


In [7]:
# def active_fallow_cropland_mask(aoi, historic_date_start, historic_date_end, current_date_start, current_date_end, crop_window, min_crop_months, min_active_crop_months):
def active_fallow_cropland_mask(aoi, historic_date_start, historic_date_end, current_date_start, current_date_end, crop_window, min_crop_months):
    
    current_date_start = ee.Date(current_date_start)
    current_date_end = ee.Date(current_date_end)
    
    crop_monthly_col = binary_crop_image_col(aoi, historic_date_start, historic_date_end)

    # Filter crop_monthly to current window
    current_crop_monthly_col = crop_monthly_col.filterDate(current_date_start, current_date_end)
    
    # Get maximum number consecutive months a a pixel was cropland in current window
    current_consective_crop_months_count = get_consecutive_crop_count(current_crop_monthly_col, crop_window)
    
    # current_consective_crop_months_count = current_crop_monthly_col.sum().rename("current_crop_months").toInt()

    # Mask pixels classified as cropland for more than min_active_crop_months as active cropland
    active_cropland_mask = current_consective_crop_months_count.gte(min_crop_months).rename("active_cropland").toByte()

    # Fallow Cropland = Potential Cropland AND NOT Active Cropland (and within aoi)
    fallow_cropland_mask = potential_cropland_mask(aoi, historic_date_start, historic_date_end, crop_window, min_crop_months).And(active_cropland_mask.Not()).rename("fallow").toByte()

    # Class image: 0 other, 1 fallow, 2 active
    classified = (ee.Image(0).where(fallow_cropland_mask, 1).where(active_cropland_mask, 2)
        .rename("class").toByte().clip(aoi))

    # Remove pixels classified as other
    return classified.updateMask(classified.neq(0))

### Class Duration/Intensity visualisation

#### Active intensity (green strength)
For **active pixels**, intensity is:

- `active_intensity = crop-month count in last 12 months` (0..12)

#### Fallow intensity (red strength)
For **fallow pixels**, intensity is:

- `months_since_crop = (current_end_month) - (last_crop_month)`

`last_crop_month` is calculated by multiplying `is_crop` by a per-month index and taking the max.

Then we cap fallow intensity at `fallow_cap_months` for stable visualization.


In [8]:
# Get the most recent month that is the end of a sliding window of length crop_window where the pixel was crop in >= min_crop_months months.
def last_sustained_crop_month_index(crop_monthly_col, historic_date_start, crop_window, min_crop_months):
    historic_date_start = ee.Date(historic_date_start)

    crop_monthly_col_size = crop_monthly_col.size()
    crop_list = crop_monthly_col.toList(crop_monthly_col_size)

    n_windows = crop_monthly_col_size.subtract(crop_window).add(1)

    # Identify crop band name once (defensive)
    first = ee.Image(crop_monthly_col.first())
    crop_band = ee.String(first.bandNames().get(0))

    def window_end_sustained_index(i):
        i = ee.Number(i)

        # Window images i..i+crop_window-1
        window_ic = ee.ImageCollection.fromImages(crop_list.slice(i, i.add(crop_window)))
        win_sum = window_ic.select([crop_band]).sum().rename("win_sum")

        # Window end image for timestamp
        end_img = ee.Image(crop_list.get(i.add(crop_window).subtract(1)))
        end_date = ee.Date(end_img.get("system:time_start"))

        # Month index (1-based)
        end_index = end_date.difference(historic_date_start, "month").toInt().add(1)

        # Valid sustained cropping if win_sum >= min_crop_months
        valid = win_sum.gte(min_crop_months)

        # Image contains index where valid, masked elsewhere
        return (ee.Image.constant(end_index)
                .rename("last_sustained_crop_index")
                .toInt()
                .updateMask(valid)
                .set("system:time_start", end_date.millis())
               )

    sustained_end_index_col = ee.ImageCollection.fromImages(
        ee.List.sequence(0, n_windows.subtract(1)).map(window_end_sustained_index)
    )

    # Most recent sustained window end index per pixel
    return sustained_end_index_col.max().rename("last_sustained_crop_index").toInt()


# def active_fallow_cropland_duration(aoi, historic_date_start, historic_date_end, current_date_start, current_date_end, crop_window, min_crop_months, min_active_crop_months, min_active_duration):
def active_fallow_cropland_duration(aoi, historic_date_start, historic_date_end, current_date_start, current_date_end, crop_window, min_crop_months, min_active_duration):
    historic_date_start = ee.Date(historic_date_start)
    historic_date_end = ee.Date(historic_date_end)
    current_date_start = ee.Date(current_date_start)
    current_date_end = ee.Date(current_date_end)

    int_current_date_end = current_date_end
    int_current_date_start = int_current_date_end.advance(-min_active_duration, "month")

    # Get isCrop collection
    crop_monthly_col = binary_crop_image_col(aoi, historic_date_start, historic_date_end)

    # Get number of months for which each pixel was cropland in the specified months for active duration
    crop_recent_col = crop_monthly_col.filterDate(int_current_date_start, int_current_date_end)
    active_cropland_duration = crop_recent_col.sum().rename("active_duration").toInt()

    # Mask to only active pixels
    active_fallow_cropland = active_fallow_cropland_mask(
        aoi, 
        historic_date_start, historic_date_end, 
        current_date_start, current_date_end, 
        crop_window, min_crop_months
        )

    # Clip active intensity to active cropland mask
    active_cropland_duration_masked = (active_cropland_duration.updateMask(active_fallow_cropland.eq(2)).clip(aoi))

   # Last sustained crop ----
    last_sustained_index = last_sustained_crop_month_index(
        crop_monthly_col, historic_date_start, crop_window, min_crop_months
    )

    # End index for the intensity reference date
    end_date_index = int_current_date_end.difference(historic_date_start, "month").toInt().add(1)

    # Months since last sustained crop window ended
    months_since_sustained_crop = (
        ee.Image.constant(end_date_index)
        .subtract(last_sustained_index)
        .rename("months_since_sustained_crop")
        .toInt()
    )

    # Guard: only pixels that have ever had a sustained window
    has_sustained_history = last_sustained_index.gt(0)

    # Mask to fallow pixels AND sustained-history pixels
    fallow_cropland_duration_masked = (
        months_since_sustained_crop
        .updateMask(active_fallow_cropland.eq(1))
        .updateMask(has_sustained_history)
        .clip(aoi)
    )

    return active_cropland_duration_masked, fallow_cropland_duration_masked


### Define aoi and parameters

Replace the aoi geometry below with:
- a drawn geometry in geemap, OR
- paste the bounding box for your aoi

**IMPORTANT:** Dynamic World is global, but processing large AOIs at 10m can be heavy. Chose a small aoi.


In [9]:
# Example AOI
aoi = ee.Geometry.Rectangle([27.75120,-15.8775, 27.9214,-15.7321]) # Mazabuka
# aoi = ee.Geometry.Rectangle([12.8918, 47.9002, 12.9543,  47.8657])
# aoi = ee.Geometry.Rectangle([40.5121,37.0099, 40.6247,37.0736])

# Long-term history window (e.g., 5 years)
historic_date_start="2020-01-01"
historic_date_end="2025-12-31"

# Current classification window (e.g., now, or last 3 months of a season)
current_date_start = "2025-06-01"
current_date_end = "2025-12-31"

# Sliding window rule for potential cropland
crop_window = 4 # window length in months
min_crop_months = 3 # minimum crop-months within any crop window

# Active cropland threshold within current window
min_active_crop_months = 3

# Months to count crop presence for active duration
min_active_duration = 12
fallow_cap_months = 24 


### Visualisation using geemap

Displays:
- Dynamic World Image
- Sentinel 2 Image
- Potential Cropland (0/1)
- Active & Fallow Cropland (1/2)
- Active duration (0–12)
- Fallow duration (0–X)


In [10]:
# Visualise original Dynamic World image and Sentinel-2 for inspection.
def visualise_original_dw_image(current_date_start, current_date_end, aoi, m):
    col_filter = ee.Filter.And(
        ee.Filter.bounds(aoi),
        ee.Filter.date(current_date_start, current_date_end),
    )

    dw_col = ee.ImageCollection('GOOGLE/DYNAMICWORLD/V1').filter(col_filter)
    s2_col = ee.ImageCollection('COPERNICUS/S2_HARMONIZED').filter(col_filter)

    # Link DW and S2 source images.
    linked_col = dw_col.linkCollection(s2_col, s2_col.first().bandNames())

    # Get example DW image with linked S2 image.
    linked_image = ee.Image(linked_col.first())

    # Create a visualization that blends DW class label with probability.
    # Define list pairs of DW LULC label and color.
    CLASS_NAMES = [
        'water',
        'trees',
        'grass',
        'flooded_vegetation',
        'crops',
        'shrub_and_scrub',
        'built',
        'bare',
        'snow_and_ice',
    ]

    VIS_PALETTE = [
        '419bdf',
        '397d49',
        '88b053',
        '7a87c6',
        'e49635',
        'dfc35a',
        'c4281b',
        'a59b8f',
        'b39fe1',
    ]

    # Create an RGB image of the label (most likely class) on [0, 1].
    dw_rgb = (
        linked_image.select('label')
        .visualize(min=0, max=8, palette=VIS_PALETTE)
        .divide(255)
    )

    # Get the most likely class probability.
    top1_prob = linked_image.select(CLASS_NAMES).reduce(ee.Reducer.max())

    # Create a hillshade of the most likely class probability on [0, 1]
    top1_prob_hillshade = ee.Terrain.hillshade(top1_prob.multiply(100)).divide(255)

    # Combine the RGB image with the hillshade.
    dw_rgb_hillshade = dw_rgb.multiply(top1_prob_hillshade)

    # Display the Dynamic World visualization with the source Sentinel-2 image.
    # m = geemap.Map()
    # m.set_center(20.6729, 52.4305, 12)
    m.add_layer(
        linked_image.clip(aoi),
        {'min': 0, 'max': 3000, 'bands': ['B4', 'B3', 'B2']},
        'Sentinel-2',
        False
    )
    m.add_layer(
        dw_rgb_hillshade.clip(aoi),
        {'min': 0, 'max': 0.65},
        'Dynamic World V1 - label hillshade',
        False
    )

In [11]:

def visualise_all(m, aoi, historic_date_start, historic_date_end, current_date_start, current_date_end, crop_window, min_crop_months, min_active_crop_months, min_active_duration, fallow_cap_months):
    if m.user_roi is not None:
        aoi = m.user_roi

    # Potential Cropland Binary Image
    potential_crop_land = potential_cropland_mask(
        aoi, 
        historic_date_start, historic_date_end, 
        crop_window, min_crop_months)

    # Active and Fallow Cropland Classification
    active_fallow_cropland = active_fallow_cropland_mask(
        aoi, 
        historic_date_start, historic_date_end, 
        current_date_start, current_date_end, 
        crop_window, min_crop_months
        )

    # Active and Fallow Cropland Duration
    active_cropland_duration, fallow_cropland_duration = active_fallow_cropland_duration(
        aoi, 
        historic_date_start, historic_date_end, 
        current_date_start, current_date_end, 
        crop_window, min_crop_months, 
        min_active_duration
        )

    m.centerObject(aoi, 12)

    m.remove_colorbar()
    m.remove_colorbar()

    # Dynamic World and Sentinel-2 Images
    visualise_original_dw_image(current_date_start, current_date_end, aoi, m)

    # Potential Cropland
    potential_vis = {"min": 0, "max": 1, "palette": ["#bdbdbd", "#5ee498"]} 
    m.addLayer(potential_crop_land.clip(aoi), potential_vis, "Potential Cropland", False)

    # Active and Fallow Cropland
    active_fallow_vis = {'min': 1, 'max': 2, 'palette':['orange','green']}
    m.addLayer(active_fallow_cropland, active_fallow_vis, "Cropland", False)

    # Active Cropland Duration
    active_crop_vis = {'min': 0, 'max': min_active_duration, 'palette':['white','green']}
    m.addLayer(active_cropland_duration, active_crop_vis, 'Active Cropland')
    m.add_colorbar(vis_params=active_crop_vis, 
                   label='Active cropland duration (months)', layer_name='Active Cropland')

    # Fallow Cropland Duration
    fallow_vis = {'min': 0,'max': fallow_cap_months, 'palette':['orange', 'red']}
    m.addLayer(fallow_cropland_duration, fallow_vis, 'Fallow')
    m.add_colorbar(vis_params=fallow_vis, label='Fallow duration (months)', layer_name='Fallow')

    return m

In [14]:
m = geemap.Map(basemap="HYBRID")

visualise_all(m, aoi, historic_date_start, historic_date_end, current_date_start, current_date_end, crop_window, min_crop_months, min_active_crop_months, min_active_duration, fallow_cap_months)

Map(center=[-15.804807858266097, 27.836300000001017], controls=(WidgetControl(options=['position', 'transparen…

In [13]:
if m.user_roi is not None:
    visualise_all(m, aoi, '2018-01-01', '2023-12-31', '2023-10-01', '2023-12-31', crop_window, min_crop_months, min_active_crop_months, min_active_duration)

NameError: name 'm' is not defined